In [2]:
%load_ext jupyternotify
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import time

import re
import string
import librosa
import librosa.display
import IPython.display as ipd

from model_wrapper import ModelWrapper

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import grid_search
from sklearn.grid_search import ParameterGrid

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
SAMPLE_RATE = 22050

In [6]:
%%notify
df_train = pd.read_csv("train.csv")
train = df_train.values

<IPython.core.display.Javascript object>

In [ ]:
N_train = train.shape[0]
NUM_SAMPLES = train.shape[1]-1

X_train = train[:,:-1]
y_train = train[:,-1]